AkHQ: http://localhost:8082  
Spark-Master http://localhost:8083  
Spark-Worker-1 http://localhost:8084

https://github.com/aehrc/pathling/tree/issue/452/lib/python#python-api-for-pathling

# Try Connect To Test FHIR Kafka

In [ ]:
!python --version

In [ ]:
import findspark
findspark.init()
findspark.find()

In [ ]:
appName = "Kafka, Spark and FHIR Data"
master = "spark://spark-master:7077"
#master = "local[*]"
kafka_topic = "fhir.post-gateway-kdb"

In [ ]:
from pyspark.sql import SparkSession
from pathling.etc import find_jar

spark = SparkSession.builder \
    .appName(appName) \
    .master(master) \
    .config('spark.jars', find_jar()) \
    .config('spark.ui.port','0') \
    .getOrCreate()

In [ ]:
# https://spark.apache.org/docs/latest/structured-streaming-kafka-integration.html
# default for startingOffsets is "latest", but "earliest" allows rewind for missed alerts    
df = spark \
  .readStream  \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka1:19092") \
  .option("subscribe", kafka_topic) \
  .option("startingOffsets", "earliest") \
  .load()

In [ ]:
df.printSchema()

In [ ]:
import os
os.getenv("PYSPARK_SUBMIT_ARGS")

In [ ]:
query = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
          .writeStream \
          .queryName("gettable") \
          .format("memory") \
          .start()

# close connection after 30 seconds
query.awaitTermination(30)

In [ ]:
kafka_data = spark.sql("select * from gettable")
kafka_data.show()
type(kafka_data)

In [ ]:
pd_df = kafka_data.toPandas()
pd_df

# Bring Pathling into the game

In [ ]:
from pathling.r4 import bundles

resources = bundles.from_json(kafka_data, 'value')

In [ ]:
patients = bundles.extract_entry(spark, resources, 'Patient')
encounter = bundles.extract_entry(spark, resources, 'Encounter')
condition = bundles.extract_entry(spark, resources, 'Condition')

In [ ]:
patients.printSchema()

In [ ]:
patients.select("id", "birthDate", "gender", "address.postalCode").toPandas()

In [ ]:
encounter.printSchema()

In [ ]:
encounter.select("id", "subject.reference", "serviceType.coding.code", "period.start", "period.end").toPandas()

In [ ]:
condition.printSchema()

In [ ]:
condition.select("id", "encounter.reference", "code.coding.code").toPandas()